**Mohammad Zhalechian**
PS2, Q3

In [1]:
#-----------------------------------------------------------------------------
#-------------------------------Library--------------------------------------
#-----------------------------------------------------------------------------
import numpy as np
import time
import scipy as sp
import scipy.stats as st
import pandas as pd
from IPython.core.display import display, HTML
from collections import defaultdict

### Question 3 - Data Cleaning

In [ ]:
# Read data from the source and data cleaning
cols = ['SEQN','RIDAGEYR','RIDRETH3','DMDEDUC2','DMDMARTL',
        'RIDSTATR', 'SDMVPSU', 'SDMVSTRA', 'WTMEC2YR', 'WTINT2YR']

col_names=['unique ids','age','race and ethnicity','education','marital status',
           'exam_status','masked variance pseudo', 'masked variance pseudo-stratum',
           '2 year MEC exam weight','2 year interview weight']

files=['G','H','I','J']
demo_df=pd.DataFrame()

for i in range(1,5):
    read_path=r'https://wwwn.cdc.gov/Nchs/Nhanes/'+\
          str(2010+(2*i-1))+'-'+str(2010+2*i)+'/DEMO_'+files[i-1]+'.XPT'
    df = pd.read_sas(read_path)[cols]
    df.columns = col_names
    df["cohort"] = np.ones(len(df))*i
    #df=df.convert_dtypes()
    demo_df = demo_df.append(df).reset_index(drop=True)

demo_df.to_pickle('demo.pkl')


cols=['SEQN','OHDDESTS']+[
    'OHX0'+str(i)+'TC' for i in range(1,10)]+ [
    'OHX'+str(i)+'TC' for i in range(10,33)]+[
    'OHX0'+str(i)+'CTC' for i in range(2,10)]+[
    'OHX'+str(i)+'CTC' for i in range(10,32)]

cols.remove('OHX16CTC')
cols.remove('OHX17CTC')

col_names=['unique ids','status']+[
    'tooth Count #'+str(i) for i in range(1,10)]+ [
    'tooth Count #'+str(i) for i in range(10,33)]+[
    'coronal cavities #'+str(i) for i in range(2,10)]+[
    'coronal cavities #'+str(i) for i in range(10,32)]

col_names.remove('coronal cavities #16')
col_names.remove('coronal cavities #17')

files=['G','H','I','J']
oral_df=pd.DataFrame()

for i in range(1,5):
    read_path=r'https://wwwn.cdc.gov/Nchs/Nhanes/'+str(2010+(2*i-1))+'-' \
          +str(2010+2*i)+'/OHXDEN_'+files[i-1]+'.XPT'
    df = pd.read_sas(read_path)[cols]
    df.columns = col_names
    df["cohort"] = np.ones(len(df))*i
    #df=df.convert_dtypes()
    oral_df = oral_df.append(df).reset_index(drop=True)

oral_df.to_pickle('oral.pkl')

#merge two dataset
cols_reg=['tooth Count #'+str(i) for i in range(1,10)] + [
          'tooth Count #'+str(i) for i in range(10,33)]

df_ohx = pd.merge(demo_df[['unique ids','age','race and ethnicity']],
                  oral_df[['unique ids']+cols_reg], 
                  on= 'unique ids' , how = 'left')

#drop NA
df_ohx = df_ohx.dropna()
df_ohx.to_pickle('df_ohx.pkl')

